In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
 
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")
 
plt.rcParams['figure.figsize'] = 15, 5

In [ ]:
   
AMD = yf.Ticker("AMD")
 
# getting the historical market data
AMD_values = AMD.history(start="2020-01-01")
 
# Plotting the close price of AMD
AMD_values[['Close']].plot()

In [ ]:
AMD_values['rolling_av'] = AMD_values['Close'].rolling(10).mean()
 
# plotting 10-day rolling average value with the closing value
AMD_values[['Close','rolling_av']].plot()

In [ ]:

# Function to plot ACF and PACF
def plot_acf_pacf(timeseries):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 7))
    plot_acf(timeseries, ax=ax1, lags=75)
    plot_pacf(timeseries, ax=ax2, lags=75)
    plt.show()
 
 
# Plotting ACF and PACF of the closing value time series
plot_acf_pacf(AMD_values['Close'])

In [ ]:
   
#creating the model
MA_model = ARIMA(endog=AMD_values['Close'], order=(0, 0, 55))
 
#fitting data to the model
results = MA_model.fit()
 
#summary of the model
print(results.summary())

In [ ]:
#prediction data
start_date = '2023-12-15'
end_date   = '2024-06-25'
AMD_values['prediction'] = results.predict(start=start_date, end=end_date)
 
#printing last 14 values of the prediction with original and rolling avg value
print(AMD_values[['Close','rolling_av','prediction']].tail(14))

In [ ]:
# Forecast future values
# Forecast future closing prices
forecast_steps = 30  # Forecasting for the next 30 days
forecast_index = pd.date_range(start=AMD_values['Close'].index[-1], periods=forecast_steps+1, freq='D')[1:]  # Generate datetime index for forecast
forecast = results.forecast(steps=forecast_steps)
 
# plotting the end results
AMD_values[['Close','rolling_av','prediction']].plot()
plt.plot(forecast_index, forecast, color='red', label='Forecast')